# Tech Challenge - Fase 3

## Integrantes do Grupo 22
* Matheus Alves da Silva - matheusa761@gmail.com
* Alexandre Pantalena Yoshimatsu - alexandre.yoshimatsu@virgo.inc
* Guilherme Duarte de Andrade Tavares - guilherme.tavares@virgo.inc



# O PROBLEMA

No Tech Challenge desta fase, você precisa executar o fine-tuning de um foundation model (Llama, BERT, MISTRAL etc.), utilizando o dataset "The AmazonTitles-1.3MM". O modelo treinado deverá:


*   Receber perguntas com um contexto obtido por meio do arquivo json
“trn.json” que está contido dentro do dataset.
*   A partir do prompt formado pela pergunta do usuário sobre o título do
produto, o modelo deverá gerar uma resposta baseada na pergunta do
usuário trazendo como resultado do aprendizado do fine-tuning os
dados da sua descrição.



# Fluxo de trabalho atualizado

## 1. Escolha do Dataset:

Descrição: o The AmazonTitles-1.3MM consiste em consultas textuais
reais de usuários e títulos associados de produtos relevantes encontrados na
Amazon e suas descrições, medidos por ações implícitas ou explícitas dos
usuários.

## 2. Preparação do Dataset:

Faça o download do dataset AmazonTitles-1.3MM e utilize o arquivo
“trn.json”. Nele, você utilizará as colunas “title” e “content”, que contêm título e
descrição respectivamente. Prepare os prompts para o fine-tuning garantindo
que estejam organizados de maneira adequada para o treinamento do modeloescolhido. Limpe e pré-processe os dados conforme necessário para o modelo
escolhido.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

path = "/content/drive/MyDrive/Colab Notebooks/trn.json"

df_trn = pd.read_json(path, lines=True)

In [ ]:
# Substitui strings vazias por NaN na coluna 'title', para facilitar a detecção de valores ausentes
df_trn['title'].replace('', np.nan, inplace=True)

# Substitui strings vazias por NaN na coluna 'content', pelo mesmo motivo acima
df_trn['content'].replace('', np.nan, inplace=True)

# Remove linhas do DataFrame onde as colunas 'title' ou 'content' possuem valores ausentes (NaN)
df_trn.dropna(subset=['title', 'content'], inplace=True)

# Exibe as 10 primeiras linhas do DataFrame após a limpeza
df_trn.head(10)


<ipython-input-3-734a52ff3699>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_trn['title'].replace('', np.nan, inplace=True)
<ipython-input-3-734a52ff3699>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

,uid,title,content,target_ind,target_rel
0,0000031909,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
3,0001360000,Mog's Kittens,Judith Kerr&#8217;s best&#8211;selling adventu...,"[146, 147, 148, 149, 495]","[1.0, 1.0, 1.0, 1.0, 1.0]"
7,0000031895,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 2-8 years. Perfect f...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
12,000100039X,The Prophet,"In a distant, timeless place, a mysterious pro...","[329, 330, 331, 332, 333, 334, 335, 336, 337, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
13,0001473905,Rightly Dividing the Word,--This text refers to thePaperbackedition.,"[181, 182, 307, 380, 381]","[1.0, 1.0, 1.0, 1.0, 1.0]"
15,0001516035,Worship with Don Moen [VHS],Worship with Don Moen [VHS],[422],[1.0]
16,0001837397,Autumn Story Brambly Hedge,"""!the most research-crammed fantasy ever set b...","[423, 424, 425, 427, 430, 438, 439, 440]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
17,0000791156,Spirit Led-Moving By Grace In The Holy Spirit'...,You can flow effortlessly and powerfully in th...,"[441, 442, 443, 444, 445, 446]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
18,0001714384,The Very Bad Bunny (Beginner Series),"By Marilyn Sadler, Illustrated by Roger Bollen","[483, 484, 485, 486, 487]","[1.0, 1.0, 1.0, 1.0, 1.0]"
22,0001983415,Nice for Mice,Jill Barklem was born in Epping in 1951. After...,[512],[1.0]


In [ ]:
# Função para transformar as colunas 'title' e 'content' em um formato de conversas
def apply_conversations(row):
    conv = [
        {"from": "human", "value": row['title']},
        {"from": "gpt", "value": row['content']}
    ]
    return conv

# Aplica a função a cada linha do DataFrame e cria uma nova coluna 'conversations' com o formato processado
df_trn["conversations"] = df_trn.apply(apply_conversations, axis=1)


In [ ]:
df_trn[["conversations"]].head(5)

,conversations
0,"[{'from': 'human', 'value': 'Girls Ballet Tutu..."
3,"[{'from': 'human', 'value': 'Mog's Kittens'}, ..."
7,"[{'from': 'human', 'value': 'Girls Ballet Tutu..."
12,"[{'from': 'human', 'value': 'The Prophet'}, {'..."
13,"[{'from': 'human', 'value': 'Rightly Dividing ..."


In [ ]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
# Instala o pacote Unsloth
!pip install unsloth

# Remove qualquer versão existente do Unsloth e instala a versão mais recente diretamente do repositório
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 164.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 111.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
grpcio-

Found existing installation: unsloth 2024.12.4
Uninstalling unsloth-2024.12.4:
  Successfully uninstalled unsloth-2024.12.4
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-s0eqbqwo
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-s0eqbqwo
  Resolved https://github.com/unslothai/unsloth.git to commit df808d074d307d396b8c04640ef51375d68f2ef0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.12.4-py3-none-any.whl size=173562 sha256=70fb6f8c1076c0540e12b44c09b75f12a56ab64651c35ee9acf602737708b3f6
  Stored in directory: /tmp/pip-ephem-wheel-cache-5taylbub/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


In [ ]:
from unsloth import FastLanguageModel
import torch

# Configurações do modelo
max_seq_length = 2048  # Define o comprimento máximo da sequência
dtype = None  # Detecta automaticamente o tipo de dado, ou especifica Float16/Bfloat16 dependendo do hardware
load_in_4bit = True  # Ativa quantização em 4 bits para reduzir o uso de memória

# Lista de modelos pré-quantizados para 4 bits, otimizados para download rápido e menor consumo de memória
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",
    "unsloth/Mistral-Small-Instruct-2409",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",
    "unsloth/Llama-3.2-1B-bnb-4bit",
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
]

# Carrega o modelo e o tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",  # Nome do modelo escolhido
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token="hf_...",  # Inclua o token se estiver utilizando modelos restritos
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Define o rank (dimensão) das matrizes LoRA. Valores mais altos permitem maior capacidade de aprendizado,
             # mas consomem mais memória. Valores típicos: 8, 16, 32, etc.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,  # Fator de escala aplicado às atualizações feitas pelas camadas LoRA.
                      # Controla a influência das atualizações LoRA no modelo principal. Típico: 1x a 2x o rank.

    lora_dropout = 0, # Taxa de dropout para as atualizações LoRA. Evita overfitting ao desativar
                      # aleatoriamente algumas conexões durante o treinamento. 0 significa sem dropout (otimizado).

    bias = "none",    # Controla se os termos de bias serão treinados.
                      # "none" congela os termos de bias (não treináveis).
                      # "all" treina todos os biases do modelo.
                      # "lora_only" treina apenas os biases dentro dos módulos LoRA.
    use_gradient_checkpointing="unsloth",  # Reduz o uso de memória para contextos mais longos. Opção "unsloth" é otimizada.
    random_state=3407,  # Define a seed para reprodutibilidade durante o treinamento.
    use_rslora=False,  # Desativa o uso de Rank Stabilized LoRA (opcional para maior estabilidade no aprendizado).
    loftq_config=None,  # Configuração para LoftQ (não utilizada neste caso).
)

Unsloth 2024.12.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from unsloth.chat_templates import get_chat_template

# Configura o tokenizer para utilizar o template de chat do modelo "llama-3.1"
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

# Função para formatar os prompts de entrada no formato esperado pelo modelo
def formatting_prompts_func(examples):
    # Extrai as conversas do exemplo fornecido
    convos = examples["conversations"]
    # Aplica o template de chat em cada conversa, sem tokenizar ou adicionar prompts de geração
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    # Retorna um dicionário com o texto formatado
    return {"text": texts}



In [ ]:
from unsloth.chat_templates import standardize_sharegpt
# Importa o módulo Dataset da biblioteca datasets para manipular o conjunto de dados
from datasets import Dataset

# Cria um dataset a partir do DataFrame pandas já carregado (df_trn)
dataset = Dataset.from_pandas(df_trn)

# Exibe o conteúdo da coluna 'conversations' para o quinto registro do dataset
dataset[5]["conversations"]


[{'from': 'human', 'value': 'Worship with Don Moen [VHS]'},
 {'from': 'gpt', 'value': 'Worship with Don Moen [VHS]'}]

In [ ]:
from unsloth.chat_templates import standardize_sharegpt

# Padroniza o dataset no formato ShareGPT para garantir compatibilidade com o modelo
dataset = standardize_sharegpt(dataset)

# Aplica a função de formatação dos prompts em todo o dataset, processando em lotes
dataset_format = dataset.map(formatting_prompts_func, batched=True)

Standardizing format:   0%|          | 0/685 [00:00<?, ? examples/s]

Map:   0%|          | 0/685 [00:00<?, ? examples/s]

In [ ]:
dataset_format[5]['text']

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWorship with Don Moen [VHS]<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nWorship with Don Moen [VHS]<|eot_id|>'

## 3. Chamada do Foundation Model
Importe o foundation model que será utilizado e faça um teste apresentando o resultado atual do modelo antes do treinamento (para que se obtenha uma base de análise após o fine-tuning), e então será possível avaliar a diferença do resultado gerado.

In [ ]:
from unsloth.chat_templates import get_chat_template

# Configura o tokenizer para o template do modelo "llama-3.1"
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

# Ativa o modo de inferência do modelo, otimizando para execução 2x mais rápida
FastLanguageModel.for_inference(model)

# Define as mensagens de entrada simulando uma conversa
messages = [
    {"role": "user", "content": "What is Nice for Mice?"},
]

# Prepara as entradas no formato esperado pelo modelo, com tokenização e prompt de geração
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,  # Necessário para permitir a geração de respostas
    return_tensors="pt",
).to("cuda")

# Gera a saída com o modelo, configurando parâmetros como número máximo de tokens e temperatura
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=64,
    use_cache=True,
    temperature=1.5,  # Controla a criatividade na geração (valores maiores = mais variabilidade)
    min_p=0.1,        # Filtra tokens de baixa probabilidade
)

# Decodifica e exibe a resposta gerada pelo modelo
tokenizer.batch_decode(outputs)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is Nice for Mice?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nNice for mice is a brand of mouse killers that is based in the United States, specifically in the state of South Carolina.<|eot_id|>']

##4. Execução do Fine-Tuning

Execute o fine-tuning do foundation model selecionado (por exemplo, BERT, GPT, Llama) utilizando o dataset preparado.
Documente o processo de fine-tuning, incluindo os parâmetros utilizados e qualquer ajuste específico realizado no modelo.

In [ ]:
# Training the model
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Desativa empacotamento, útil para sequências curtas (pode acelerar treinamento em alguns casos)
        args = TrainingArguments(
        per_device_train_batch_size = 2,  # Tamanho do batch processado por GPU. Ajustável conforme a memória da GPU.
        gradient_accumulation_steps = 4,  # Acumula gradientes em múltiplas iterações para simular um batch maior.
        warmup_steps = 5,  # Número de etapas iniciais para aumentar a taxa de aprendizado gradualmente.
        # num_train_epochs = 1, # Opcional: Define o número de épocas completas de treinamento.
        max_steps = 60,  # Número total de passos de treinamento (limita a duração do treinamento).
        learning_rate = 2e-4,  # Taxa de aprendizado que controla a magnitude das atualizações dos pesos.
        fp16 = not is_bfloat16_supported(),  # Usa precisão de 16 bits para economizar memória se BF16 não for suportado.
        bf16 = is_bfloat16_supported(),  # Usa precisão BF16 se a GPU suportar (moderna, como NVIDIA A100).
        logging_steps = 1,  # Frequência de registro dos logs de métricas (perda, tempo, etc.).
        optim = "adamw_8bit",  # Otimizador escolhido. `adamw_8bit` usa precisão reduzida para economizar memória.
        weight_decay = 0.01,  # Regularização que penaliza grandes pesos, ajudando a evitar overfitting.
        lr_scheduler_type = "linear",  # Tipo de ajuste da taxa de aprendizado (linear reduz progressivamente até 0).
        seed = 3407,  # Semente aleatória para reprodutibilidade.
        output_dir = "outputs",  # Diretório onde os modelos e logs serão salvos.
        report_to = "none",  # Define para onde os resultados serão enviados (ex.: WandB, TensorBoard).
    ),
)

Map (num_proc=2):   0%|          | 0/685 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
from unsloth.chat_templates import train_on_responses_only

# Ajusta o treinador para focar apenas nas respostas geradas pelo modelo
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",  # Identifica a seção do usuário na entrada
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",  # Identifica a seção de resposta do modelo
)


Map:   0%|          | 0/685 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWorship with Don Moen [VHS]<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nWorship with Don Moen [VHS]<|eot_id|>'

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                             \n\nWorship with Don Moen [VHS]<|eot_id|>'

In [ ]:
# Exibe as estatísticas atuais de memória da GPU
gpu_stats = torch.cuda.get_device_properties(0)  # Obtém as propriedades da GPU
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)  # Memória máxima reservada em GB
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)  # Memória total disponível em GB

# Exibe o nome da GPU e informações de memória
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")


GPU = Tesla T4. Max memory = 14.748 GB.
2.832 GB of memory reserved.


In [ ]:
# Inicia o treinamento do modelo usando o treinador configurado
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 685 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,3.213800
2,3.734900
3,3.367900
4,3.242800
5,3.422700
6,3.301200
7,3.292700
8,2.806900
9,3.249300
10,2.808400


In [ ]:
# Exibe as estatísticas finais de memória e tempo após o treinamento
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)  # Memória máxima reservada em GB
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)  # Memória utilizada exclusivamente para LoRA
used_percentage = round(used_memory / max_memory * 100, 3)  # Porcentagem da memória total utilizada
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)  # Porcentagem da memória usada para LoRA

# Exibe o tempo de execução do treinamento
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime'] / 60, 2)} minutes used for training.")

# Exibe estatísticas de pico de uso de memória
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

190.2445 seconds used for training.
3.17 minutes used for training.
Peak reserved memory = 7.68 GB.
Peak reserved memory for training = 4.848 GB.
Peak reserved memory % of max memory = 52.075 %.
Peak reserved memory for training % of max memory = 32.872 %.


##5. Geração de Respostas

Configure o modelo treinado para receber perguntas dos usuários.
O modelo deverá gerar uma resposta baseada na pergunta do usuário e nos dados provenientes do fine-tuning, incluindo as fontes fornecidas.

In [ ]:
from unsloth.chat_templates import get_chat_template

# Configura o tokenizer para utilizar o template do modelo "llama-3.1"
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

# Ativa o modo de inferência do modelo, otimizando para execução 2x mais rápida
FastLanguageModel.for_inference(model)

# Define as mensagens de entrada simulando uma interação com o usuário
messages = [
    {"role": "user", "content": "What is Nice for Mice?"},
]

# Prepara as entradas no formato esperado pelo modelo, incluindo tokenização e prompt de geração
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,  # Necessário para permitir a geração de respostas
    return_tensors="pt",
).to("cuda")

# Gera a saída usando o modelo, com configurações para controle de criatividade e filtro
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=64,  # Define o número máximo de novos tokens gerados
    use_cache=True,     # Usa cache para otimizar a geração
    temperature=0.2,    # Controla a criatividade; valores baixos geram saídas mais previsíveis
    min_p=0.1,          # Filtro para descartar tokens de baixa probabilidade
)

# Decodifica e exibe a resposta gerada pelo modelo
tokenizer.batch_decode(outputs)


["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is Nice for Mice?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nA delightful children's book, illustrated by the famous Beatrix Potter. The story is about a group of mice who live in a house in the countryside. They are very tidy and organized, and they have a very good system for keeping their house clean. The mice are very proud of their home and are always trying to"]

##Treinando modelo alternativo

In [ ]:
from unsloth import FastLanguageModel

# Define o nome do modelo base
base_model_name = "unsloth/Llama-3.2-3B-Instruct"

# Carrega o modelo base e o tokenizador com configurações semelhantes ao anterior
model_base, tokenizer_base = FastLanguageModel.from_pretrained(
    model_name=base_model_name,
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

# Configura um modelo alternativo com LoRA ajustada
model_alternative = FastLanguageModel.get_peft_model(
    model_base,
    r=64,  # **Mudou:** Aumenta o rank para 64, permitindo maior capacidade de aprendizado.
    target_modules=["q_proj", "k_proj", "v_proj"],  # **Mudou:** Ajusta apenas os módulos específicos (redução de escopo).
    lora_alpha=128,  # **Mudou:** Aumenta o impacto das camadas LoRA no modelo principal.
    lora_dropout=0.2,  # **Mudou:** Introduz um dropout de 20% para prevenir overfitting.
    bias="none",  # Mantém os termos de bias congelados (não treináveis).
    use_gradient_checkpointing="unsloth",  # Otimiza o uso de memória para modelos grandes.
    random_state=3407,  # Semente para reprodutibilidade.
)



==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.2.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.12.4 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq

# Configura o treinador para o modelo alternativo
trainer_alternative = SFTTrainer(
    model=model_alternative,  # Usa o modelo alternativo com parâmetros ajustados
    tokenizer=tokenizer,  # Tokenizador permanece o mesmo
    train_dataset=dataset,  # Dataset de treinamento não foi alterado
    dataset_text_field="text",  # Campo de texto no dataset
    max_seq_length=max_seq_length,  # Mantém o comprimento máximo das sequências
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),  # Colator de dados permanece igual
    dataset_num_proc=2,  # Número de processos para pré-processamento dos dados
    packing=False,  # Sem empacotamento de sequências
    args=TrainingArguments(
        per_device_train_batch_size=4,  # **Mudou:** Dobrou o batch size por dispositivo (2 -> 4), usando mais memória.
        gradient_accumulation_steps=2,  # **Mudou:** Reduziu para acumular gradientes mais frequentemente (4 -> 2).
        warmup_steps=10,  # **Mudou:** Aumentou o número de passos de warmup para o dobro (5 -> 10).
        max_steps=120,  # **Mudou:** Dobrou o número máximo de passos de treinamento (60 -> 120).
        learning_rate=1e-4,  # **Mudou:** Reduziu pela metade a taxa de aprendizado (2e-4 -> 1e-4) para ajustes mais finos.
        fp16=not is_bfloat16_supported(),  # Usa FP16 se BF16 não for suportado.
        bf16=is_bfloat16_supported(),  # Usa BF16 se suportado.
        logging_steps=10,  # **Mudou:** Reduziu a frequência de logging (1 -> 10 passos).
        optim="adamw_8bit",  # Otimizador permanece eficiente em memória.
        weight_decay=0.05,  # **Mudou:** Dobrou o peso de decaimento para evitar overfitting (0.01 -> 0.05).
        lr_scheduler_type="linear",  # Mantém o agendador linear.
        seed=1234,  # **Mudou:** Alterou a semente para resultados diferentes (3407 -> 1234).
        output_dir="outputs_alternative",  # **Mudou:** Novo diretório para salvar resultados.
        report_to="none",  # Logs desativados para ferramentas externas.
    ),
)



Map (num_proc=2):   0%|          | 0/685 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# Ajusta o treinador alternativo para focar apenas nas respostas geradas pelo modelo
trainer_alternative = train_on_responses_only(
    trainer_alternative,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",  # Identifica o segmento de instrução do usuário
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",  # Identifica o segmento de resposta do assistente
)

# Decodifica e exibe os IDs de entrada do dataset do treinador alternativo
print(tokenizer.decode(trainer_alternative.train_dataset[5]["input_ids"]))

# Substitui os valores -100 por um espaço e decodifica os labels do dataset
space = tokenizer(" ", add_special_tokens=False).input_ids[0]
print(tokenizer.decode([space if x == -100 else x for x in trainer_alternative.train_dataset[5]["labels"]]))

# Obtém as propriedades da GPU e calcula o uso inicial de memória
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# Realiza o treinamento usando o treinador alternativo
trainer_stats = trainer_alternative.train()

# Calcula o uso de memória durante o treinamento
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

# Exibe estatísticas de tempo e uso de memória após o treinamento
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime'] / 60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")



Map:   0%|          | 0/685 [00:00<?, ? examples/s]

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Worship with Don Moen [VHS]<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Worship with Don Moen [VHS]<|eot_id|>
                                             

Worship with Don Moen [VHS]<|eot_id|>
GPU = Tesla T4. Max memory = 14.748 GB.
7.742 GB of memory reserved.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 685 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 2
\        /    Total batch size = 8 | Total steps = 120
 "-____-"     Number of trainable parameters = 25,690,112


Step,Training Loss
10,3.524900
20,3.213700
30,2.945500
40,2.920100
50,2.854600
60,2.891700
70,2.814900
80,2.824300
90,2.777700
100,2.712700


475.4904 seconds used for training.
7.92 minutes used for training.
Peak reserved memory = 7.742 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 52.495 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model_alternative)

messages = [
    {"role": "user", "content": "What is the content of Nice for Mice?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model_alternative.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                                      temperature = 0.5, min_p = 0.1)
response_alternative = tokenizer.batch_decode(outputs)
print("Resposta do modelo alternativo:", response_alternative)


Resposta do modelo alternativo: ['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the content of Nice for Mice?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe story is set in a large house in the countryside, where a group of mice live in the walls. The mice are all very different, with different personalities and skills. They are all very intelligent, and they have a very sophisticated system of communication. They live in a large, beautiful house with many rooms, and']
